# Automated Marketing Lead Screening Pipelines using Snowflake Cortex
Efficiently managing and qualifying marketing leads is crucial for business growth, but manually processing large volumes of form submissions can be time-consuming and inconsistent. In this notebook, you'll use Cortex AISQL to systematically analyze, cleanse, and enrich incoming marketing leads captured from web forms.

### Context
This is an example of how Snowflake's own marketing analytics team leverages Snowflake's Cortex AISQL to automate lead qualification processes. The system processes form submissions from marketing events and webinars, filtering out spam content, scoring leads based on their fit with the Ideal Customer Profile (ICP), and categorizing them by job seniority.

In this notebook, we will leverage multiple AISQL functions to build an intelligent lead management workflow that:
- Filters out spam and irrelevant submissions
- Scores lead quality based on ICP alignment
- Categorizes leads by seniority level
- Generates personalized outreach content

The primary business value is creating a more efficient and intelligent lead management workflow that saves manual effort, enables sales and marketing teams to prioritize high-potential prospects, and allows for personalized outreach at scale.

##

## Step 1: Set up your environment and data

Let's begin by running the query below. It sets the correct role and warehouse for this session and creates a new schema. It also creates and populates the `MARKETING_FORM_DATA` and `COMPANY_INFO` tables with sample data for our analysis.

In [ ]:
USE ROLE SNOWFLAKE_LEARNING_ROLE;

-- Use the existing database, schema and warehouse
USE DATABASE SNOWFLAKE_LEARNING_DB;
USE WAREHOUSE SNOWFLAKE_LEARNING_WH;

SET schema_name = CONCAT(current_user(), '_MARKETING_LEAD_PIPELINE');
CREATE SCHEMA IF NOT EXISTS IDENTIFIER($schema_name);
USE SCHEMA IDENTIFIER($schema_name);

/*--
• File format and stage creation
--*/

CREATE OR REPLACE FILE FORMAT csv_ff 
  TYPE = 'csv'
  SKIP_HEADER = 1;

CREATE OR REPLACE STAGE s3load
  COMMENT = 'Quickstarts S3 Stage Connection'
  URL = 's3://sfquickstarts/misc/aisql/marketing_lead_pipelines/'
  FILE_FORMAT = csv_ff;

/*--
• Raw zone table build 
--*/
CREATE OR REPLACE TABLE marketing_form_data
(
	PERSON_ID VARCHAR(16777216),
	FIRST_NAME VARCHAR(16777216),
	LAST_NAME VARCHAR(16777216),
	TITLE VARCHAR(16777216),
	COMPANY VARCHAR(16777216)
);

CREATE OR REPLACE TABLE company_info
(
  COMPANY VARCHAR(16777216),
  DESCRIPTION VARCHAR(16777216)
);

/*--
• Raw zone table load 
--*/

COPY INTO marketing_form_data
FROM @s3load/marketing_form_data.csv
ON_ERROR = CONTINUE;

COPY INTO company_info
FROM @s3load/company_info.csv
ON_ERROR = CONTINUE;

-- Setup completion note
SELECT 'Setup is complete' AS note;

#### Overview of `marketing_form_data` table

In [ ]:
SELECT * FROM marketing_form_data LIMIT 10;

## Step 2: Filter out spam and irrelevant submissions

As a first step in our lead qualification process, we need to filter out spam, test entries, and irrelevant form submissions. We'll use the `AI_FILTER` function to automatically identify legitimate business leads from valid personas with companies, excluding junk data, spam, jokes, or student submissions.

In [ ]:
----------------------------------------------------------------------------------------------------
--- filter out Spam Records
----------------------------------------------------------------------------------------------------

CREATE OR REPLACE TEMP TABLE MARKETING_LEADS_FILTERED AS
SELECT d.*
FROM marketing_form_data d
WHERE AI_FILTER(
  'We are a B2B SAAS company, and this is the text entered to a demand gen form on our website to register for a marketing event. Please confirm if this is a form fill that appears to be legitimate data from a valid persona with a company, without any junk, spam, scams, jokes, or nonsensical entries that cannot be worked by sales. Please do not include students:' 
  || CONCAT_WS(' ',
      'First Name:', d.first_name,
      'Last Name:', d.last_name,
      'Job Title:', d.title,
      'Company:', d.company
    )
);

SELECT * FROM MARKETING_LEADS_FILTERED LIMIT 100;

## Step 3: Score leads based on Ideal Customer Profile (ICP)

Now, let's classify each filtered lead into quality categories: "High", "Medium", "Low", or "Poor" based on their alignment with our Ideal Customer Profile. We'll use `AI_CLASSIFY` to systematically evaluate leads based on their decision-making role, company characteristics, and business need for data and AI services.

In [ ]:
CREATE OR REPLACE TEMP TABLE MARKETING_LEADS_W_QUALITY AS
SELECT 
  d.*,
  AI_CLASSIFY(
    CONCAT_WS(' ',
      'Job Title:', d.title,
      'Company:', d.company
    ),
    [
      {
        'label': 'High',
        'description': 'The lead has a decision-making role, relevant persona, and comes from a company well-aligned with B2B SAAS target industries and size. They must have a clear business need for data and AI services.'
      },
      {
        'label': 'Medium',
        'description': 'The lead is a good fit but may lack full decision-making authority or strong company characteristics. However, they should have influence or future potential.'
      },
      {
        'label': 'Low',
        'description': 'The lead has minimal alignment, lacks decision-making power, or is from a less relevant company. The persona or company factors do not strongly align with B2B SAAS ICP.'
      },
      {
        'label': 'Poor',
        'description': 'The lead has no alignment with B2B SAAS ICP. Their role and company characteristics are irrelevant. A form fill that appears to contain test accounts, junk, spam, scams, jokes, or nonsensical entries should be labeled as poor.'
      }
    ],
    {
      'task_description': 'We are a B2B SAAS company. Return a classification for the Ideal Customer Profile of this lead based on the text entered to a demand gen form on our website to register for a marketing event.'
    }
  ) AS classification_output_raw,
  classification_output_raw:labels[0]::TEXT AS leads_quality
FROM MARKETING_LEADS_FILTERED d;

SELECT * FROM MARKETING_LEADS_W_QUALITY LIMIT 10;

## Step 4: Categorize leads by job seniority

To further enhance our lead qualification, we can classify leads based on their job seniority level. This helps sales teams understand the decision-making authority and tailor their approach accordingly. We'll use `AI_CLASSIFY` to categorize job titles into seniority groups ranging from C-level executives to individual contributors.

In [ ]:
CREATE OR REPLACE TEMP TABLE MARKETING_LEADS_W_SENIORITY AS
SELECT 
  d.*,
  AI_CLASSIFY(
    CONCAT_WS(' ', 'Job Title:', d.title),
    [
      {
        'label': 'CXO',
        'description': 'Any title that is a company C-level executive or founder.'
      },
      {
        'label': 'VP+',
        'description': 'Any title that is a company executive below C-level like a VP, President, or managing director.'
      },
      {
        'label': 'Director',
        'description': 'Any title that is director level or head of a department.'
      },
      {
        'label': 'Manager',
        'description': 'Any title that relates to managers or team leads.'
      },
      {
        'label': 'IC',
        'description': 'Any title that relates to an individual contributor.'
      },
      {
        'label': 'Junk',
        'description': 'Any title seems like junk, spam, scams, jokes, or nonsensical entries that cannot be worked by sales'
      }
    ],
    {
      'task_description': 'We are a B2B SAAS company. Use this data to classify job titles into seniority groupings. Consider all parts of the title and be careful of how the meaning changes based on parentheses or other punctuation.'
    }
  ) AS seniority_classification_raw,
  seniority_classification_raw:labels[0]::TEXT AS seniority_level
FROM MARKETING_LEADS_W_QUALITY d;

SELECT * FROM MARKETING_LEADS_W_SENIORITY LIMIT 10;

## Step 5: Generate personalized outreach content

For high-quality leads, we can leverage company information to create personalized outreach messages. By joining lead data with company insights, we can use `AI_COMPLETE` to draft customized emails that reference specific company details and use cases, significantly improving response rates.

In [ ]:
WITH high_quality_leads AS (
  SELECT 
    ml.*,
    ci.description AS company_description,
  FROM MARKETING_LEADS_W_SENIORITY ml
  LEFT JOIN company_info ci ON ml.company = ci.company
  WHERE ml.leads_quality = 'High'
)
SELECT 
  first_name,
  last_name,
  title,
  company,
  leads_quality,
  seniority_level,
  AI_COMPLETE(
    'llama4-maverick',
    PROMPT(
      'We are a B2B SAAS company focusing on data and analytics platform. This is a form filled by our potential prospect. Can you help me draft an outreach email to ask if they have any interest in how data and analytics might supercharge their business. Please only return the email draft and nothing else. Here is the filled form: {0}; Here is the company info: {1}',
      CONCAT_WS(' ',
        'First Name:', first_name,
        'Last Name:', last_name,
        'Job Title:', title,
        'Company:', company
      ),
      COALESCE(company_description, '')
    )
  ) AS personalized_email
FROM high_quality_leads
LIMIT 5;

## Key Takeaways

* **End-to-End Automation**: You can chain Cortex AI functions together (`AI_FILTER` -> `AI_CLASSIFY` -> `AI_COMPLETE`) to build a comprehensive lead qualification pipeline entirely within Snowflake.
* **Intelligent Filtering**: AI-powered spam detection eliminates manual review of irrelevant submissions, ensuring your sales team focuses on legitimate prospects.
* **ICP Scoring**: Automated lead scoring based on Ideal Customer Profile criteria helps prioritize sales efforts on the highest-potential opportunities.
* **Personalized Outreach**: By combining lead data with company intelligence, you can generate customized messaging that significantly improves response rates.
* **Scalable Process**: This automated workflow can handle thousands of leads while maintaining consistency and quality in qualification decisions.

## Additional Resources

* [Documentation: Cortex AI SQL Functions](https://docs.snowflake.com/en/user-guide/snowflake-cortex/aisql)